In [1]:
import numpy as np
from scipy  import stats
import math


\begin{align*}
L_i &= \{C_i, C_{i+1}\} \quad for \quad i = 1,\ldots, T-1\\
\psi_{L_i}(C_i, C_{i+1}) &= P(C_{i+1} \mid C_i)\\
M_{i}  &= \{C_i, Z_{i,1}, Z_{i,2}, \ldots, Z_{i,n}\} \quad for \quad i =1,\ldots, T\\
\psi_{M_i}(C_i, Z_{i,1}, Z_{i,2}, \ldots, Z_{i,n})&= \prod_{j = 1}^n P(Z_{i,j} \mid C_i)\\
N_{i,j} &= \{Z_{i,j}, X_{i,j}\} \quad for \quad i = 1, \ldots, T, j = 1,\ldots, n\\
\psi_{N_{i,j}}(Z_{i,j}, X_{i,j}) &= P(X_{i,j} \mid Z_{i,j})
\end{align*}

\begin{align*}
 M_1  \to L_1 \to \ldots \to M_{i+1} \to L_{i+1} \ldots \to M_{T}
\end{align*}

In [326]:
class Variables:
    def __init__(self) -> None:
        self.alpha = 0.9
        self.beta = 0.2
        self.gamma = 0.1
        self.lamb0 = 1
        self.lamb1 = 5
        self.rates = [self.lamb0, self.lamb1]
        self.prob_C1 = np.array([0,0,1])
        self.Gamma = np.array([[1-self.gamma, 0, self.gamma], #(Ci, Ci_plus1)
                          [0, 1-self.gamma, self.gamma], 
                          [self.beta/2, self.beta/2, 1-self.beta]])
        self.alpha_C = np.array([1-self.alpha, self.alpha, 0.5])
        self.T = 10
        self.n = 2
        self.Z_given_C = np.array([[self.alpha, 1-self.alpha],[1-self.alpha, self.alpha], [0.5, 0.5]])  #(C, Z)
        

    def sim_data(self):
        ck = np.arange(3)   
    
        C_transition = [
            stats.rv_discrete(values=(ck,self.Gamma[0,])),#P(C_t |C_{t-1} = 0)
            stats.rv_discrete(values=(ck,self.Gamma[1,])),#P(C_t |C_{t-1} = 1)
            stats.rv_discrete(values=(ck,self.Gamma[2,])),#P(C_t |C_{t-1} = 2)
        ]
        
        C = np.zeros(self.T, np.int64)
        C[0] = 2
        for i in range(self.T-1):
            C[i+1] = C_transition[C[i]].rvs()
        self.C = C
        
        self.Z = stats.bernoulli([self.Z_given_C[c,1] for c in C]).rvs(size=[self.n,self.T])
        self.X = stats.poisson(np.where(self.Z, self.lamb1, self.lamb0)).rvs() #(n, T)
        return 
    
    #def initialize_potentials(self):
    #    self.Psi_Nijs = np.array([stats.poisson.pmf(k = self.X, mu = self.lamb0), #All of the potentials
    #                        stats.poisson.pmf(k = self.X, mu = self.lamb1)]) #(Z, n, T)
    #    self.Psi_M0 = np.einsum("nCZ, C -> nCZ", np.array([self.Z_given_C]*self.n), self.prob_C1) #(n, C, Z)
    #    self.Psi_Mi_slice =   np.array([self.Z_given_C]*self.n)  #only a slice of the psi_Mi #(n, C, Z)
    #    self.Psi_Li_slice = self.Gamma #only a slice of psi_L

        
        
    #def d_Nij_to_Mi(self):
    #    self.delta_Nij_to_Mi = self.Psi_Nijs #Z,n,T
    #
    #def update_Mi(self):
   #     self.Psi_M0_new = np.einsum("nCZ, Zn -> nCZ", self.Psi_M0, self.Psi_Nijs[:,:,0])
    #    self.Psi_Mi_new = np.einsum("nCZ, ZnT -> TnCZ", self.Psi_Mi_slice, self.Psi_Nijs[:,:,1:])

    def find_P_X_given_C(self):
        self.P_X_given_Z = np.array([stats.poisson.pmf(k = self.X, mu = self.lamb0), #All of the potentials
                            stats.poisson.pmf(k = self.X, mu = self.lamb1)]) #(Z, n, T)
        
        self.P_X_given_C = np.einsum("ZnT, CZ -> TnC",self.P_X_given_Z, self.Z_given_C) #(Time, n, C)

    def upwards_passing(self):
        delta_Mi_to_Li = np.empty(shape = (self.T-1, 3)) #(Time, Ci)
        delta_Li_to_Mi_plus = np.empty(shape = (self.T-1, 3)) #(C_{i+1})
        belief_Li = np.empty(shape = (self.T, 3, 3)) #(T, C_i, C_{i+1})
    
    
        for i in range(self.T-1):
            #finding message Ci, Zi -> Ci+, Ci+
            if i == 0:
                delta_Mi_to_Li[i] = np.prod(self.P_X_given_C[i,:,:], axis = 0)*self.prob_C1 #C1
            else:
                delta_Mi_to_Li[i] = np.prod(self.P_X_given_C[i,:,:], axis = 0)*delta_Li_to_Mi_plus[i-1] #Ci

            #normalizing
            delta_Mi_to_Li[i] /= np.sum(delta_Mi_to_Li[i])

            #Finding belief Li
            belief_Li[i] = np.einsum("cC,c -> cC", self.Gamma, delta_Mi_to_Li[i]) #c old (C_{i}), C new (C_{i+1})
            #normalizing
            belief_Li[i] /= np.sum(belief_Li[i])
            
            #finding message Ci Ci+ -> Ci+, Zi+
            delta_Li_to_Mi_plus[i] = np.sum(belief_Li[i], axis=0) #Summing out c old
            
            #normalizing
            delta_Li_to_Mi_plus[i] /= np.sum(delta_Li_to_Mi_plus[i])

        self.delta_Mi_to_Li = delta_Mi_to_Li
        self.delta_Li_to_Mi_plus = delta_Li_to_Mi_plus
        self.belief_Li = belief_Li
        return 
    
    def backwards_passing(self):
        delta_Mi_plus_to_Li = np.empty(shape= (self.T-1, 3)) #T, C
        delta_Li_to_Mi = np.empty(shape= (self.T -1, 3)) #T, C
        updated_belief_Li = np.empty(shape = (self.T-1, 3, 3)) #T, c old (C_{i}), C new (C_{i+1})       

        for i in range(self.T-1):
            #find downwards message Mi+ -> Li
            j = self.T -1 - i -1 #finding opposite direvtion

            if i == 0: #then j = T-2, aka the last entry index in messages
                delta_Mi_plus_to_Li[j] = np.prod(self.P_X_given_C[j+1,:,:], axis = 0) #(C)
            else:
                delta_Mi_plus_to_Li[j] = np.prod(self.P_X_given_C[j+1,:,:], axis = 0)*delta_Li_to_Mi[j+1] #(C)


            #normalize
            delta_Mi_plus_to_Li[j] /= np.max(delta_Mi_plus_to_Li[j])

            #find belief
            updated_belief_Li[j] = np.einsum("cC, C -> cC", np.einsum("cC, c -> cC", self.Gamma, self.delta_Mi_to_Li[j]), delta_Mi_plus_to_Li[j])
            
            #normalize
            updated_belief_Li[j] /= np.sum(updated_belief_Li[j])
            
            # find message Li -> Mi

            delta_Li_to_Mi[j] = np.einsum("cC,C -> c", self.Gamma, delta_Mi_plus_to_Li[j]) #Summing out

            #normalize
            delta_Li_to_Mi[j] /= np.sum(delta_Li_to_Mi[j])
            print(delta_Mi_plus_to_Li[[j]])
            print(delta_Li_to_Mi[[j]])


        self.updated_belief_Li = updated_belief_Li
        self.delta_Li_to_Mi = delta_Li_to_Mi
        self.delta_Mi_plus_to_Li = delta_Mi_plus_to_Li
        
        return 
        
   

variables = Variables()
variables.sim_data()
variables.X = np.array([[1]*variables.T]*variables.n)

In [327]:
variables.X

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [328]:
#generate data
variables.find_P_X_given_C() #time, C
variables.upwards_passing()
variables.backwards_passing()

[[1.         0.04025948 0.36038869]]
[[0.66828979 0.05159925 0.28011096]]
[[1.         0.00310847 0.15105546]]
[[0.79287311 0.01551181 0.19161508]]
[[1.00000000e+00 7.87638366e-04 8.70957881e-02]]
[[0.83530051 0.00865759 0.1560419 ]]
[[1.00000000e+00 4.17275241e-04 6.73239568e-02]]
[[0.84920613 0.00665699 0.14413688]]
[[1.00000000e+00 3.15597068e-04 6.11692510e-02]]
[[0.85363152 0.0060302  0.14033828]]
[[1.00000000e+00 2.84399723e-04 5.92484331e-02]]
[[0.85502291 0.00583352 0.13914358]]
[[1.00000000e+00 2.74676097e-04 5.86484532e-02]]
[[0.85545855 0.00577195 0.1387695 ]]
[[1.00000000e+00 2.71638857e-04 5.84609944e-02]]
[[0.85559477 0.0057527  0.13865253]]
[[1.00000000e+00 2.70689847e-04 5.84024193e-02]]
[[0.85563734 0.00574669 0.13861597]]


In [289]:
variables.delta_Mi_plus_to_Li

array([[0.03752737, 0.69322032, 1.        ],
       [0.14070872, 0.1593013 , 1.        ],
       [0.28396436, 0.13663663, 1.        ],
       [0.37885776, 0.11984713, 1.        ],
       [0.01644609, 1.        , 0.64191204],
       [0.26364534, 0.38655771, 1.        ],
       [0.78004988, 0.19857947, 1.        ],
       [0.56350923, 0.33614282, 1.        ],
       [1.        , 0.02591643, 0.33893199]])

In [312]:
print(np.around(variables.delta_Mi_to_Li, decimals=2))
#print(variables.Z) #(n, T)
print(variables.X) #(n, T)
print(np.around(variables.belief_Li, decimals = 2)[7])
#probabilities of Xi given Zi
#print(variables.Psi_Nijs) #(Z, n, T)
#print(variables.updated_belief_Li)

[[0.   0.   1.  ]
 [0.02 0.21 0.77]
 [0.04 0.19 0.77]
 [0.05 0.13 0.82]
 [0.16 0.1  0.74]
 [0.02 0.41 0.57]
 [0.   0.72 0.28]
 [0.   0.87 0.13]
 [0.01 0.63 0.36]]
[[ 1  3  1  6  3  4  5  4 10  5]
 [ 1  4  5  0  1  4  4  5  0  7]]
[[0.   0.   0.  ]
 [0.   0.78 0.09]
 [0.01 0.01 0.1 ]]


In [325]:
print(np.around(variables.updated_belief_Li, decimals=2)[0])
print(np.around(variables.belief_Li, decimals=2)[0])

[[0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.73 0.27]]
[[0.  0.  0. ]
 [0.  0.  0. ]
 [0.1 0.1 0.8]]


In [329]:
def find_C_probabilites():
    C_prob = np.empty(shape = (variables.T, 3, 2)) #Time, C, the two methods
    updated_beliefs = variables.updated_belief_Li
    updated_beliefs = updated_beliefs

    for i in range(variables.T-1):
        guess_0 = np.sum(updated_beliefs[i], axis=1)
        guess_1 = np.sum(updated_beliefs[i], axis= 0)
        C_prob[i,:,0] = guess_0
        C_prob[i+1,:,1] = guess_1
    
    C_prob[0,:,1] = C_prob[0,:,0]
    C_prob[-1,:,0] = C_prob[-1,:,1]

    print(np.around(C_prob, decimals= 2))

find_C_probabilites()
variables.X

[[[0.   0.  ]
  [0.   0.  ]
  [1.   1.  ]]

 [[0.68 0.68]
  [0.   0.  ]
  [0.32 0.32]]

 [[0.89 0.89]
  [0.   0.  ]
  [0.11 0.11]]

 [[0.96 0.96]
  [0.   0.  ]
  [0.04 0.04]]

 [[0.98 0.98]
  [0.   0.  ]
  [0.02 0.02]]

 [[0.99 0.99]
  [0.   0.  ]
  [0.01 0.01]]

 [[0.99 0.99]
  [0.   0.  ]
  [0.01 0.01]]

 [[0.99 0.99]
  [0.   0.  ]
  [0.01 0.01]]

 [[0.98 0.98]
  [0.   0.  ]
  [0.02 0.02]]

 [[0.94 0.94]
  [0.   0.  ]
  [0.06 0.06]]]


array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [228]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [137]:
def upwards_passing():
    delta_Mi_to_Li = np.empty(shape = (variables.T-1, 3)) #(Time, Ci)
    delta_Li_to_Mi_plus = np.empty(shape = (variables.T-1, 3)) #(C_{i+1})
    belief_Li = np.empty(shape = (variables.T, 3, 3)) #(T, C_i, C_{i+1})
    
    
    for i in range(variables.T-1):
        #finding message Ci, Zi -> Ci+, Ci+
        if i == 0:
            delta_Mi_to_Li[i] = variables.P_X_given_C[i]*variables.prob_C1 #C1
        else:
            delta_Mi_to_Li[i] = variables.P_X_given_C[i]*delta_Li_to_Mi_plus[i-1] #Ci
        

        #normalizing
        delta_Mi_to_Li[i] /= np.sum(delta_Mi_to_Li[i])

        #Finding belief Li
        belief_Li[i] = np.einsum("cC,c -> cC", variables.Gamma, delta_Mi_to_Li[i]) #c old (C_{i}), C new (C_{i+1})
        belief_Li[i] /= np.sum(belief_Li[i])
        #finding message Ci Ci+ -> Ci+, Zi+
        delta_Li_to_Mi_plus[i] = np.sum(belief_Li[i], axis=0)
        #normalizing
        delta_Li_to_Mi_plus[i] /= np.sum(delta_Li_to_Mi_plus[i])

    return delta_Mi_to_Li, delta_Li_to_Mi_plus, belief_Li
        


In [169]:
def backwards_passing():
    delta_Mi_plus_to_Li = np.empty(shape= (variables.T-1, 3)) #T, C
    delta_Li_to_Mi = np.empty(shape= (variables.T -1, 3)) #T, C
    updated_belief_Li = np.empty(shape = (variables.T-1, 3, 3)) #T, c old (C_{i}), C new (C_{i+1})

    for i in range(variables.T-1):
        #find downwards message Mi+ -> Li
        j = variables.T -1 - i -1 #finding opposite direvtion
        if i == 0:
            delta_Mi_plus_to_Li[i] = variables.delta_Li_to_Mi_plus[j]*variables.P_X_given_C[j] #(C)
        else:
            delta_Mi_plus_to_Li[i] = variables.delta_Li_to_Mi_plus[j]*delta_Li_to_Mi[i]*variables.P_X_given_C[j] #(C)
        
        #normalize
        delta_Mi_plus_to_Li[i] /= np.max(delta_Mi_plus_to_Li[i])

        #find belief
        updated_belief_Li[i] = np.einsum("cC, C -> cC", variables.belief_Li[j], delta_Mi_plus_to_Li[i]/variables.delta_Li_to_Mi_plus[j])
        # find message Li -> Mi

        delta_Li_to_Mi[i] = np.sum(updated_belief_Li[i], axis=1)
        
        #normalize
        delta_Li_to_Mi[i] /= np.sum(delta_Li_to_Mi[i])

    return updated_belief_Li, delta_Li_to_Mi, delta_Mi_plus_to_Li
        
backwards_passing()

(array([[[9.67278499e-01, 0.00000000e+00, 8.70204697e-02],
         [0.00000000e+00, 2.12573210e-02, 3.08772003e-03],
         [3.27215013e-02, 2.02662653e-02, 2.11951066e-01]],
 
        [[9.66758614e-01, 0.00000000e+00, 2.92033582e-03],
         [0.00000000e+00, 1.15564430e-04, 7.26437152e-05],
         [3.32413860e-02, 1.59742326e-04, 7.22980135e-03]],
 
        [[9.54996163e-01, 0.00000000e+00, 1.45842588e-02],
         [0.00000000e+00, 5.16637059e-05, 5.10100333e-04],
         [4.50038373e-02, 6.96084929e-05, 4.94839978e-02]],
 
        [[9.23703465e-01, 0.00000000e+00, 6.86015641e-02],
         [0.00000000e+00, 6.10409292e-03, 1.00718544e-02],
         [7.62965350e-02, 3.43413969e-03, 4.07979890e-01]],
 
        [[8.86078705e-01, 0.00000000e+00, 2.85045759e-02],
         [0.00000000e+00, 9.67640743e-03, 7.13747334e-03],
         [1.13921295e-01, 4.96840333e-03, 2.63863728e-01]],
 
        [[1.40334713e-01, 0.00000000e+00, 5.05755735e-02],
         [0.00000000e+00, 1.13191062e-01,

[1.         0.04152359 0.30205926]
[[[9.67278499e-001 0.00000000e+000 8.70204697e-002]
  [0.00000000e+000 2.12573210e-002 3.08772003e-003]
  [3.27215013e-002 2.02662653e-002 2.11951066e-001]]

 [[3.80397588e+291 2.11658057e-110 5.76527396e-114]
  [9.33847987e+252 5.83911122e+252 5.17435671e-114]
  [6.21839860e+175 9.48749771e+222 1.17143144e+200]]

 [[2.20898020e+213 1.07852282e+229 1.67953988e+243]
  [1.86189562e-004 1.84841356e-004 3.93269204e-028]
  [5.30581254e+180 9.46694842e+218 1.55446670e+142]]

 [[1.35905790e+200 6.66126231e-115 3.13676357e+174]
  [7.29537384e+175 2.15922561e+185 3.94701974e+252]
  [8.52623465e+257 6.49160949e+174 8.92899224e+252]]

 [[6.46336856e+170 3.93258379e-028 5.36532746e-109]
  [3.65415130e+291 1.98952897e+248 4.25434125e+179]
  [2.49865394e+262 3.79226711e+214 3.93268423e-028]]

 [[5.73189017e+194 1.96900296e+248 4.89321283e+252]
  [2.30628298e+160 1.35727532e+248 8.52623465e+257]
  [3.93266696e-028 7.65404890e+189 1.67953971e+243]]

 [[1.86420307e-00

ValueError: could not broadcast input array from shape (9,3) into shape (3,)

In [45]:
#Find temporary clique belief at Mi
temp_Mi = np.einsum("ZnT, CZ-> TCZ", variables.P_x_given_z, variables.Z_given_C) #(Time, C, Z)
temp_Mi[0] = np.einsum("CZ, C-> CZ", temp_Mi[0], variables.prob_C1) #(C, Z)
#upwards passing
delta_Mi_to_Li = np.empty(shape= (variables.T-1, 3)) #T, C
delta_Li_to_Mi_plus = np.empty(shape= (variables.T , 3)) #T, C
for i in range(variables.T-1):
    if i == 0:
        delta_Mi_to_Li[i] = np.sum(temp_Mi[i], axis = 1) #(C)
    else:
        delta_Mi_to_Li[i] = np.einsum("C, CZ -> C", delta_Li_to_Mi_plus[i], temp_Mi[i]) #(C)
    #print("deltaMi is \n", delta_Mi_to_Li[i])
    #normalize
    delta_Mi_to_Li[i] /= np.max(delta_Mi_to_Li[i])

    delta_Li_to_Mi_plus[i] = np.einsum("c, cC -> C", delta_Mi_to_Li[i], variables.Gamma) #C
    
    #normalize
    delta_Li_to_Mi_plus[i] /= np.max(delta_Li_to_Mi_plus[i])

    #print("delta li is \n", delta_Li_to_Mi_plus[i])

delta_Li_to_Mi_plus[-1 ] = np.einsum("c, cC -> C", delta_Mi_to_Li[-1], variables.Gamma)
delta_Li_to_Mi_plus[-1] /= np.max(delta_Li_to_Mi_plus[-1])
#variables.temp_Mi = np.sum(variables.delta_Nij_to_Mi, axis=2) #(time, C)
#print(variables.temp_Mi)


In [75]:
#backwards passing
delta_Mi_to_Li_minus = np.empty(shape= (variables.T-1, 3)) #T, C
delta_Li_to_Mi = np.empty(shape= (variables.T , 3)) #T, C

for i in range(variables.T-1)[::-1]:
    print(i)
    #----- find message ----#
    delta_Mi_to_Li_minus[i] = np.einsum("c, cC -> C", belief_Mi[i], variables.Gamma) #C
    
    #normalize
    delta_Mi_to_Li_minus[i] /= np.max(delta_Mi_to_Li_minus[i])*delta_Li_to_Mi_plus[i]

    #--- Update belief
    belief_Li[i] = [[delta_Mi_to_Li_minus[i][C_plus]*delta_Mi_to_Li[i][C]*variables.Gamma[C_plus][C] for C_plus in range(3)] for C in range(3)]
    belief_Li[i] /= sum(belief_Li[i])
    #----- find message ----#
    if i == 0:
        delta_Li_to_Mi[i] = np.sum(temp_Mi[i], axis = 1) #(C)
    else:
        delta_Li_to_Mi[i] = np.einsum("C, CZ -> C", delta_Mi_to_Li_minus[i], temp_Mi[i]) #(C)
    #print("deltaMi is \n", delta_Mi_to_Li[i])
    #normalize
    delta_Li_to_Mi[i] /= np.max(delta_Li_to_Mi[i])*delta_Mi_to_Li[i]
    #--- Update belief
   
    belief_Mi[i] = sum([delta_Li_to_Mi[i][:]*delta_Li_to_Mi_plus[i-1][:]*temp_Mi[i][:,z] for z in range(2)])
    print(belief_Mi[i])
    belief_Mi[i] /= sum(belief_Mi)
   


8
[nan nan nan]
7
[nan nan nan]
6
[nan nan nan]
5
[nan nan nan]
4
[nan nan nan]
3
[nan nan nan]
2
[nan nan nan]
1
[nan nan nan]
0
[       nan        nan 0.07495214]


C:\Users\otto\AppData\Local\Temp\ipykernel_15448\2325405696.py:23: RuntimeWarning: invalid value encountered in divide
  delta_Li_to_Mi[i] /= np.max(delta_Li_to_Mi[i])*delta_Mi_to_Li[i]


In [74]:
belief_Mi

array([[       nan,        nan, 0.13141598],
       [       nan,        nan, 0.15683029],
       [       nan,        nan, 0.05398623],
       [       nan,        nan, 0.02480699],
       [       nan,        nan, 0.02650979],
       [       nan,        nan, 0.06300151],
       [       nan,        nan, 0.02035382],
       [       nan,        nan, 0.01131753],
       [       nan,        nan, 0.01716385],
       [0.66622576, 0.00554201, 0.12142051]])

In [55]:
list(range(4)[::-1])

[3, 2, 1, 0]

In [101]:
#Clique factors
psi_Li = prob_Ci_given_Ci_  #dimensions (T-1, 3, 3)
#psi_M1 = np.einsum(cz, ) prob_Zij_given_Ci*
psi_Mi = np.array([np.prod([prob_Zij_given_Ci[c, z] for z in [0,1]]) for c in [0,1,2]])
psi_Nij = np.array([[[prob_Xij_given_Zij(z = zij, x = get_Xij(time = i, neuron = j)) for zij in [0,1] ] for j in range(n)] for i in range(T)]) #deterministic, shape = (T, n, 2)

In [ ]:




#Finding all Xij_given_Zij values in advance
array_Xij_given_Zij_val = [[[prob_Xij_given_Zij(z = zij, x = get_Xij(time = i, neuron = j)) for zij in range(2)] for j in range(n)] for i in range(T)]

#Sum of each sublist
sums = np.sum(array_Xij_given_Zij_val, axis=2)
# Normalize each sublist
array_Xij_given_Zij_norm = array_Xij_given_Zij_val / sums[:,:, np.newaxis] #(T, neuron, Z)

#finding product of Xij's
array_prod_Xij_given_Zij = np.prod(array_Xij_given_Zij_norm, axis = 1) #(Time, Z_value) multplied probabilites for neurons that share time and Z-value
sums1 = np.sum(array_prod_Xij_given_Zij, axis=1)
array_prod_Xij_given_Zij_norm = array_prod_Xij_given_Zij/ sums1[:,np.newaxis]

list_factor_val = np.empty(shape = (2*T -1, 3)) #2*(number of time steps) minus 1 and


In [82]:
#####Mij to Ci
delta_Zi_to_Ci = np.array([[sum([array_prod_Xij_given_Zij_norm[time, z]*prob_Zij_given_Ci[c_val, z] for z in range(2)]) for c_val in range(3)] for time in range(T)]) #summing out Z's accross cval and time
print(delta_Zi_to_Ci) #shape (time, c_val)

[[0.10000195 0.89999805 0.5       ]
 [0.76134059 0.23865941 0.5       ]
 [0.8986604  0.1013396  0.5       ]
 [0.8986604  0.1013396  0.5       ]]


In [90]:
C1 = np.array([delta_Zi_to_Ci[0, c_val]*prob_C1[c_val] for c_val in range(3)])
print(C1)  #(C1)

[0.  0.  0.5]


In [89]:
L1 = np.array([[C1[c1]*prob_Ci_given_Ci_[c1, c2] for c1 in range(3)] for c2 in range(3)])
print(L1) #(C2, C1)

[[0.   0.   0.05]
 [0.   0.   0.05]
 [0.   0.   0.4 ]]


In [93]:
C2 = [sum([L1[c2, c1]*delta_Zi_to_Ci[1, c2] for c1 in range(3)]) for c2 in range(3)]
print(C2) #C2

[0.038067029596617785, 0.011932970403382209, 0.19999999999999998]


In [94]:
L2 = np.array([[C2[c2]*prob_Ci_given_Ci_[c2, c3] for c2 in range(3)] for c3 in range(3)])

In [70]:
array_Xij_given_Zij_norm[0,0,0]*array_Xij_given_Zij_norm[0,1,0]

2.43179798020126e-06

In [37]:
prod_Zi_Ci(10, 0, 1) + prod_Zi_Ci(10, 0, 0)

0.0018133701107229535

In [43]:
1%2

1